In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import backend as K

In [2]:
url = "https://github.com/brendenlake/omniglot/raw/master/python/images_background.zip"
path_to_zip = tf.keras.utils.get_file("images_background.zip", origin=url, extract=True)
path_to_zip

'C:\\Users\\pouya\\.keras\\datasets\\images_background_extracted'

In [3]:
IMG_SIZE = 105  # Image size for resizing
BATCH_SIZE = 32
EPOCHS = 50

In [4]:
def load_image(image_path):
    if not os.path.isfile(image_path):
        print(f"Skipped: {image_path} is a directory.")
        return None  # Skip directories
    
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # Normalize to [0, 1]
    return image

def get_image_paths_and_labels(base_dir):
    image_paths = []
    labels = []
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.png') or file.endswith('.jpg'):
                image_paths.append(os.path.join(root, file))
                labels.append(root.split('\\')[-2])  # Character as label
    return image_paths, labels

# Update the path to your dataset directory
base_dir = os.path.join(path_to_zip,'images_background')
base_dir = os.path.join(base_dir, 'train')
image_paths, labels = get_image_paths_and_labels(base_dir)
print(f"Total images: {len(image_paths)}, Total labels: {len(set(labels))}")


Total images: 17600, Total labels: 27


In [5]:
from sklearn.preprocessing import LabelEncoder

def prepare_data(image_paths, labels):
    images = []
    valid_paths = []
    for path in image_paths:
        img = load_image(path)
        if img is not None:
            images.append(img)
            valid_paths.append(path)
    return np.array(images), valid_paths

X, valid_paths = prepare_data(image_paths, labels)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform([path.split('\\')[-3] for path in valid_paths])
print(y)

[ 0  0  0 ... 26 26 26]


In [6]:
def create_pairs(images, labels):
    pairs = []
    pair_labels = []
    num_classes = len(np.unique(labels))
    
    # Create pairs
    for idx in range(len(images)):
        current_img = images[idx]
        current_label = labels[idx]
        
        # Positive pair
        pos_idx = np.random.choice(np.where(labels == current_label)[0])
        pos_img = images[pos_idx]
        pairs.append([current_img, pos_img])
        pair_labels.append(1)
        
        # Negative pair
        neg_label = (current_label + np.random.randint(1, num_classes)) % num_classes
        neg_idx = np.random.choice(np.where(labels == neg_label)[0])
        neg_img = images[neg_idx]
        pairs.append([current_img, neg_img])
        pair_labels.append(0)
    
    return np.array(pairs), np.array(pair_labels)

pairs, pair_labels = create_pairs(X, y)


In [7]:
def build_feature_extractor():
    base_model = ResNet50(weights=None, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.trainable = False
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(len(set(labels)), activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

model = build_feature_extractor()
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 105, 105,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 111, 111,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 53, 53,    │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 53, 53,    │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 53, 53,    │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 55, 55,    │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 27, 27,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 27, 27,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 27, 27,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 27, 27,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 27, 27,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 27, 27,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 27, 27,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 27, 27,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 27, 27,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 27, 27,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 27, 27,    │      1,024 │ conv2_block1_3_c

 Total params: 24,119,195 (92.01 MB)

 Trainable params: 531,483 (2.03 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [8]:
class EuclideanDistance(layers.Layer):
    def __init__(self, **kwargs):
        super(EuclideanDistance, self).__init__(**kwargs)

    def call(self, inputs):
        x1, x2 = inputs
        return K.sqrt(K.sum(K.square(x1 - x2), axis=-1, keepdims=True))

In [17]:
def contrastive_loss(y_true, y_pred, margin=1.0):
    squared_pred = K.square(y_pred)  # D^2
    margin_square = K.square(K.maximum(margin - y_pred, 0))  # (margin - D)^2
    return K.mean((1 - y_true) * squared_pred + y_true * margin_square)

$L2 =\sqrt{\sum \limits _{j=1} ^{n}({x_1}_i - {x_2}_i)^2}$

In [18]:
def build_siamese_network():
    feature_extractor = build_feature_extractor()
    
    input_a = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    input_b = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    
    feat_a = feature_extractor(input_a)
    feat_b = feature_extractor(input_b)
    
    # Compute L2 distance instead of L1 for contrastive loss
    l2_distance = layers.Lambda(lambda tensors: K.sqrt(K.sum(K.square(tensors[0] - tensors[1]), axis=-1, keepdims=True)))([feat_a, feat_b])
    
    # Siamese Network
    siamese_network = models.Model(inputs=[input_a, input_b], outputs=l2_distance)
    
    # Compile with Contrastive Loss
    siamese_network.compile(optimizer='adam', loss=contrastive_loss, metrics=['accuracy'])
    
    return siamese_network


In [11]:
X_a = pairs[:, 0]
X_b = pairs[:, 1]

X_a = np.stack(X_a, axis=0)
X_b = np.stack(X_b, axis=0)

X_train_a, X_val_a, X_train_b, X_val_b, y_train, y_val = train_test_split(X_a, X_b, pair_labels, test_size=0.2, random_state=42)


In [15]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="best_model_refactor.h5",   
    save_best_only=True, 
    monitor="val_loss",
    mode="min",
    verbose=1,
)

In [19]:
siamese_network = build_siamese_network()

history = siamese_network.fit(
    [X_train_a, X_train_b], y_train,
    validation_data=([X_val_a, X_val_b], y_val),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint_callback]
)


Epoch 1/50
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - accuracy: 0.5026 - loss: 0.2838
Epoch 1: val_loss improved from inf to 0.50163, saving model to best_model_refactor.h5


880/880 ━━━━━━━━━━━━━━━━━━━━ 338s 377ms/step - accuracy: 0.5026 - loss: 0.2838 - val_accuracy: 0.4972 - val_loss: 0.5016
Epoch 2/50
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 298ms/step - accuracy: 0.4999 - loss: 0.2702
Epoch 2: val_loss did not improve from 0.50163
880/880 ━━━━━━━━━━━━━━━━━━━━ 326s 370ms/step - accuracy: 0.4999 - loss: 0.2702 - val_accuracy: 0.4972 - val_loss: 0.5018
Epoch 3/50
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step - accuracy: 0.5075 - loss: 0.2662
Epoch 3: val_loss improved from 0.50163 to 0.50120, saving model to best_model_refactor.h5


880/880 ━━━━━━━━━━━━━━━━━━━━ 324s 368ms/step - accuracy: 0.5075 - loss: 0.2662 - val_accuracy: 0.4972 - val_loss: 0.5012
Epoch 4/50
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step - accuracy: 0.5062 - loss: 0.2650
Epoch 4: val_loss did not improve from 0.50120
880/880 ━━━━━━━━━━━━━━━━━━━━ 323s 367ms/step - accuracy: 0.5062 - loss: 0.2650 - val_accuracy: 0.4972 - val_loss: 0.5014
Epoch 5/50
880/880 ━━━━━━━━━━━━━━━━━━━━ 0s 294ms/step - accuracy: 0.5013 - loss: 0.2640
Epoch 5: val_loss did not improve from 0.50120
880/880 ━━━━━━━━━━━━━━━━━━━━ 323s 367ms/step - accuracy: 0.5013 - loss: 0.2640 - val_accuracy: 0.4972 - val_loss: 0.5014
Epoch 6/50
876/880 ━━━━━━━━━━━━━━━━━━━━ 1s 296ms/step - accuracy: 0.5007 - loss: 0.2619

KeyboardInterrupt: 

In [11]:
loss, accuracy = siamese_network.evaluate([X_val_a, X_val_b], y_val)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")


29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.7996 - loss: 0.5377
Validation Loss: 0.5191612839698792, Validation Accuracy: 0.8049569129943848


In [ ]:
siamese_network.save("model_weights.h5")

In [10]:
######### inference ########


import tensorflow as tf


NUM_WAYS = 3     # 3 classes per episode
NUM_SHOTS = 2    # 2 examples per class for support set
NUM_QUERIES = 3  # 1 query example per class (you'll use your 3 new images)


# Load the trained model
siamese_net = siamese_network


# Manually selected paths for the support set (2 images per class, 3 classes)
support_image_paths = [
    # Class 0 images / Balinese
    "/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Balinese/character01/0108_01.png",
    "/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Balinese/character01/0108_02.png",

    # Class 1 images /  Bengali
    "/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Bengali/character08/0139_01.png",
    "/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Bengali/character08/0139_12.png",

    # Class 2 images / Blackfoot
    "/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Blackfoot_(Canadian_Aboriginal_Syllabics)/character04/0181_04.png",
    "/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Blackfoot_(Canadian_Aboriginal_Syllabics)/character06/0183_06.png"
]

# Corresponding labels for the support set
support_labels = [
    0, 0,  # Class 0
    1, 1,  # Class 1
    2, 2   # Class 2
]

NameError: name 'siamese_network' is not defined

In [14]:
def load_and_preprocess(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # Normalize to [0, 1]
    return image

def prepare_manual_support_set(support_image_paths, support_labels):
    support_images = [load_and_preprocess(img) for img in support_image_paths]
    support_images = np.stack(support_images)
    support_labels = np.array(support_labels)
    return support_images, support_labels

support_images, support_labels = prepare_manual_support_set(support_image_paths, support_labels)
print(f"Support Images Shape: {support_images.shape}, Labels Shape: {support_labels.shape}")


Support Images Shape: (6, 105, 105, 3), Labels Shape: (6,)


In [17]:
# Manually selected paths for the query set (3 new images to classify)
query_image_paths = [
    
    # Class 2 images / Blackfoot
   "/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Blackfoot_(Canadian_Aboriginal_Syllabics)/character04/0181_05.png",

   # Class 1 images / Bengali
   "/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Bengali/character08/0139_03.png",

   
   # Class 0 images / Balinese
   "/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Balinese/character01/0108_08.png"
   

]

def prepare_query_images(query_image_paths):
    print(query_image_paths)
    query_images = [load_and_preprocess(img) for img in query_image_paths]
    return np.stack(query_images)

query_images = prepare_query_images(query_image_paths)
print(f"Query Images Shape: {query_images.shape}")


['/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Blackfoot_(Canadian_Aboriginal_Syllabics)/character04/0181_05.png', '/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Bengali/character08/0139_03.png', '/home/amin/Desktop/fsl/datasets/omniglot_extracted/images_background/val/Balinese/character01/0108_08.png']
Query Images Shape: (3, 105, 105, 3)


In [16]:
def classify_images(support_images, support_labels, query_images, model):
    predictions = []
    for query_img in query_images:
        query_img = tf.expand_dims(query_img, axis=0)  # Expand dims for batch compatibility
        # Repeat query image for each support image
        tiled_query = np.tile(query_img, (support_images.shape[0], 1, 1, 1))
        # Predict similarity scores
        scores = model.predict([tiled_query, support_images])
        
        # Aggregate scores per class
        aggregated_scores = np.zeros(NUM_WAYS)
        for i in range(NUM_WAYS):
            aggregated_scores[i] = np.mean(scores[i * NUM_SHOTS:(i + 1) * NUM_SHOTS])
        
        predicted_class = np.argmax(aggregated_scores)
        predictions.append(predicted_class)
    return predictions

predictions = classify_images(support_images, support_labels, query_images, siamese_net)

for i, pred in enumerate(predictions):
    print(f"Image {i + 1}: Predicted Class -> {pred}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Image 1: Predicted Class -> 1
Image 2: Predicted Class -> 0
Image 3: Predicted Class -> 1
